In [1]:
import numpy as np
import pandas as pd

In [2]:
# load data
dataX=pd.read_csv("./data/trainX.csv")
dataA=pd.read_csv("./data/trainA.csv")
dataY=pd.read_csv("./data/trainY.csv")

# remove index
dataX=dataX.values[:,1:]
dataY=dataY.values[:,1:-1]
dataA=dataA.values[:,1:]

In [3]:
# split cure or not cure
data_CX=dataX[np.where(dataA==1)[0]]
dataNCX=dataX[np.where(dataA!=1)[0]]
data_CY=dataY[np.where(dataA==1)]
dataNCY=dataY[np.where(dataA!=1)]

In [4]:
from keras import models
from keras import layers

Using TensorFlow backend.
c:\users\chutzpah\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\chutzpah\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\chutzpah\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\chutzpah\anaco

In [5]:
import tensorflow as tf

In [29]:
def build_dnn_relu_adam():
  # 2. 모델의 구조를 BatchNormalization layer를 사용하여 만든다.
  X = tf.keras.layers.Input(shape=[17])
  
  H = tf.keras.layers.Dense(256)(X)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H)
  
  H = tf.keras.layers.Dense(128)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H)
  
  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H)

  H = tf.keras.layers.Dense(32)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H)

  Y = tf.keras.layers.Dense(1)(H)
  model = tf.keras.models.Model(X, Y)
  model.compile(loss='mse', optimizer='adam')
  return model

def build_dnn_tanh_adam():
  # 2. 모델의 구조를 BatchNormalization layer를 사용하여 만든다.
  X = tf.keras.layers.Input(shape=[17])
  
  H = tf.keras.layers.Dense(256)(X)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('tanh')(H)
  
  H = tf.keras.layers.Dense(128)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('tanh')(H)
  
  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('tanh')(H)

  H = tf.keras.layers.Dense(32)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('tanh')(H)
  
  Y = tf.keras.layers.Dense(1)(H)
  model = tf.keras.models.Model(X, Y)
  model.compile(loss='mse', optimizer='adam')
  return model

In [24]:
# 치료받은 사람들 중 생존시간이 100이하인 사람들만 고른다.
dataCX_100 = dataX[np.where((dataA == 1) & (dataY <= 100))[0]]
dataCY_100 = dataY[np.where((dataA == 1) & (dataY <= 100))[0]]

# 치료받지 않은 사람들 중 생존시간이 100이하인 사람들만 고른다.
dataNCX_100 = dataX[np.where((dataA == 0) & (dataY <= 100))[0]]
dataNCY_100 = dataY[np.where((dataA == 0) & (dataY <= 100))[0]]

In [31]:
# 3.데이터로 모델을 학습(FIT)합니다.
model = build_dnn_relu_adam() # 최저 loss 값 : 0.0728
model.fit(dataCX_100, dataCY_100, epochs=10000, verbose=0)
model.fit(dataCX_100, dataCY_100, epochs=10)

Epoch 1/10
264/264 [==============================] - 0s 958us/step - loss: 0.3767
Epoch 2/10
264/264 [==============================] - 0s 890us/step - loss: 0.3864
Epoch 3/10
264/264 [==============================] - 0s 811us/step - loss: 0.4003
Epoch 4/10
264/264 [==============================] - 0s 811us/step - loss: 0.2294
Epoch 5/10
264/264 [==============================] - 0s 595us/step - loss: 0.3211
Epoch 6/10
264/264 [==============================] - 0s 955us/step - loss: 0.3127
Epoch 7/10
264/264 [==============================] - 0s 856us/step - loss: 0.4251
Epoch 8/10
264/264 [==============================] - 0s 606us/step - loss: 0.3222
Epoch 9/10
264/264 [==============================] - 0s 591us/step - loss: 0.2235
Epoch 10/10
264/264 [==============================] - 0s 822us/step - loss: 0.2768


In [30]:
# 3.치료하지 않았고 생존시간이 100 이하인 환자 데이터로 모델을 학습(FIT)합니다.
model_NC = build_dnn_tanh_adam() # 최저 loss 값 : 0.0101
model_NC.fit(dataNCX_100, dataNCY_100, epochs=10000, verbose=0)
model_NC.fit(dataNCX_100, dataNCY_100, epochs=10)

Epoch 1/10
398/398 [==============================] - 0s 766us/step - loss: 0.1584
Epoch 2/10
398/398 [==============================] - 0s 651us/step - loss: 0.0771
Epoch 3/10
398/398 [==============================] - 0s 432us/step - loss: 0.0735
Epoch 4/10
398/398 [==============================] - 0s 485us/step - loss: 0.0906
Epoch 5/10
398/398 [==============================] - ETA: 0s - loss: 0.142 - 0s 430us/step - loss: 0.1256
Epoch 6/10
398/398 [==============================] - 0s 455us/step - loss: 0.0834
Epoch 7/10
398/398 [==============================] - 0s 553us/step - loss: 0.0788
Epoch 8/10
398/398 [==============================] - 0s 435us/step - loss: 0.0805
Epoch 9/10
398/398 [==============================] - 0s 417us/step - loss: 0.1073
Epoch 10/10
398/398 [==============================] - 0s 399us/step - loss: 0.1032


In [20]:
파일경로 = './data/testX.csv'
testX = pd.read_csv(파일경로)
testX=testX.values[:,1:]
print(testX.shape)

(286, 17)


In [21]:
# 치료할 경우 생존 시간을 구한다.
cured = model.predict(testX)
# 치료하지 않을 경우 생존 시간을 구한다.
non_cured = model_NC.predict(testX)

# 치료한 경우 생존 시간 > 치료하지 않은 경우 생존 시간 : 1, 
# 치료한 경우 생존 시간 > 치료하지 않은 경우 생존 시간 : 0
result = np.where(cured > non_cured, 1, 0)
print(result.shape)

(286, 1)


In [22]:
import csv 

# 이차원 리스트 
with open('result_20200829_1.csv','w', newline='') as f: 
  makewrite = csv.writer(f) 
  
  for value in result: 
    makewrite.writerow(value)


In [ ]:
"""
from openpyxl import Workbook
 
write_wb = Workbook()
 
#이름이 있는 시트를 생성
#write_ws = write_wb.create_sheet('생성시트')
 
#Sheet1에다 입력
write_ws = write_wb.active
write_ws['A1'] = 'Title'
write_ws['B1'] = 'action'
 
#행 단위로 추가
# write_ws.append([1,2,3])
 
#셀 단위로 추가
i = 1
for value in result:
  write_ws.cell(i,2, value)
  i= i+1
write_wb.save('/content/drive/My Drive/traindata/result_20200824_3.csv')
"""

"\nfrom openpyxl import Workbook\n \nwrite_wb = Workbook()\n \n#이름이 있는 시트를 생성\n#write_ws = write_wb.create_sheet('생성시트')\n \n#Sheet1에다 입력\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'Title'\nwrite_ws['B1'] = 'action'\n \n#행 단위로 추가\n# write_ws.append([1,2,3])\n \n#셀 단위로 추가\ni = 1\nfor value in result:\n  write_ws.cell(i,2, value)\n  i= i+1\nwrite_wb.save('/content/drive/My Drive/traindata/result_20200824_3.csv')\n"